In [1]:
import pandas as pd

In [8]:
# first load tables

domain_table = pd.read_csv('/Volumes/PGH-Backup/domains/IPS/test_cat/all_IPS_results.tsv', sep='\t', header=None)

display(domain_table, domain_table.shape)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,Amidase_UniRef100_UPI00068D388E|798|g__Bacillu...,5e26d200a96cb5f9ea2077e03b21700c,266,Pfam,PF01520,N-acetylmuramoyl-L-alanine amidase,58,256,9.300000e-42,T,25-07-2024,IPR002508,"N-acetylmuramoyl-L-alanine amidase, catalytic ..."
1,Amidase_UniRef100_UPI0006B0624C|678|g__Bacillu...,c9e01160fcbeb852dc6e8f1ed1eabc2d,226,Pfam,PF01520,N-acetylmuramoyl-L-alanine amidase,6,170,1.300000e-26,T,25-07-2024,IPR002508,"N-acetylmuramoyl-L-alanine amidase, catalytic ..."
2,Amidase_UniRef100_UPI0006B0624C|678|g__Bacillu...,c9e01160fcbeb852dc6e8f1ed1eabc2d,226,Pfam,PF05036,SPOR domain,192,223,1.100000e-06,T,25-07-2024,IPR007730,Sporulation-like domain
3,Amidase_UniRef100_UPI000C9E9FDF|1716|g__Vibrio...,4649943be98becffba027e8bbd34a134,572,Pfam,PF01520,N-acetylmuramoyl-L-alanine amidase,169,388,1.600000e-43,T,25-07-2024,IPR002508,"N-acetylmuramoyl-L-alanine amidase, catalytic ..."
4,Amidase_UniRef100_UPI000C9E9FDF|1716|g__Vibrio...,4649943be98becffba027e8bbd34a134,572,Pfam,PF11741,AMIN domain,33,135,4.100000e-17,T,25-07-2024,IPR021731,AMIN domain
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107338,UC118_WP_160944966.1|1551|g__Ligilactobacillus...,e016b1d9b174ec66992850fbef4c98d1,517,Pfam,PF01476,LysM domain,34,77,2.100000e-11,T,25-07-2024,IPR018392,LysM domain
1107339,UC118_WP_160998332.1|1515|g__Ligilactobacillus...,27d421e12a913112ee1d18a5bd385322,505,Pfam,PF01476,LysM domain,94,136,6.800000e-15,T,25-07-2024,IPR018392,LysM domain
1107340,UC118_WP_160998332.1|1515|g__Ligilactobacillus...,27d421e12a913112ee1d18a5bd385322,505,Pfam,PF01476,LysM domain,34,77,2.000000e-11,T,25-07-2024,IPR018392,LysM domain
1107341,UC118_WP_160998332.1|1515|g__Ligilactobacillus...,27d421e12a913112ee1d18a5bd385322,505,Pfam,PF01476,LysM domain,252,292,7.200000e-09,T,25-07-2024,IPR018392,LysM domain


(1107343, 13)

In [6]:
# Group by the enzyme name and aggregate the domain column as a list
grouped_df = domain_table.groupby(0)[12].apply(list).reset_index()


In [7]:
display(grouped_df)

,0,12
0,Amidase_UniRef100_A0A009ES59|570|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase domain]
1,Amidase_UniRef100_A0A009FUX6|744|g__Acinetobac...,"[N-acetylmuramoyl-L-alanine amidase, catalytic..."
2,Amidase_UniRef100_A0A009H4S4|567|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase domain]
3,Amidase_UniRef100_A0A009HT94|567|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase domain]
4,Amidase_UniRef100_A0A009L0R9|591|g__Acinetobac...,[N-acetylmuramoyl-L-alanine amidase domain]
...,...,...
707784,UC118_WP_253005939.1|1548|g__Ligilactobacillus...,"[Endopeptidase, NLPC/P60 domain, LysM domain, ..."
707785,UC118_WP_255820014.1|1563|g__Ligilactobacillus...,"[LysM domain, LysM domain, LysM domain, Endope..."
707786,UC118_WP_263296879.1|1554|g__Ligilactobacillus...,"[Endopeptidase, NLPC/P60 domain, LysM domain, ..."
707787,UC118_WP_263297069.1|1554|g__Ligilactobacillus...,"[LysM domain, LysM domain, LysM domain, Endope..."
